# 智能体（AI Agents）

Agent 这个词表示一个可以执行任务的实体，它可以是人类，也可以是一个程序。宽泛的讲，我们使用的任何软件都可以看作是一个 Agent，比如：

- 浏览器：你通过浏览器上网，浏览器就是一个 Agent，它可以帮助你找到你想要的信息；
- 打车软件：你通过打车软件打车，打车软件就是一个 Agent，它可以帮助你找到你想要的车；

这些既然都是 Agent，那之前为什么他们没有让 Agent 这个概念流行起来呢？因为他们没有“大脑”，或者说，他们没有“思考”的能力，他们更多的是根据预设的规则来执行任务，叫他们“工具”更合适。

比如：当你在浏览器输入一个网址然后点击跳转，他会直接跳转到目标网站，不会思考为什么这个网站会是你想要的，需不需要再找几个类似的网站给你；

直到大语言模型（Large Language Model, LLM）的出现，我们终于有了一个可以“思考”的工具，终于可以构建一个可以“思考”的 Agent 了，大家给他起了个名字，叫 AI Agent，翻译成中文就是“智能体”。以后，我们使用 Agent 来指代 AI Agent。

让 LLM 代替人的角色来思考，把那些没有“大脑”的工具用起来，就是构建 AI Agent 的思路。

构建智能体的第一步，需要先明确这个智能体要完成的任务是什么（注意，不是解决什么问题，而是完成什么任务）。




接下来，我们将逐步探索基于大模型的应用，是如何一步步从简单的Prompt应用，一步步进化到智能的Agents。中间涉及到代码的部分，我们会使用 [LangChain](https://www.langchain.com/) 和 [LangGraph](https://langchain-ai.github.io/langgraph/) 来实现。

## Chat models

[Chat models](https://python.langchain.com/docs/concepts/chat_models/) 是大语言模型应用的基础，通常来说，我们通过API来调用它们，他们接受一组消息作为输入，并返回一个消息作为输出。

LangChain 提供了标准化的接口来使用 Chat Models，可以让我们很轻松的使用和调整不同供应商所提供的模型。

运行下面的代码，需要申请 OPENAI_API_KEY，并设置环境变量，可以通过 https://github.com/chatanywhere/GPT_API_free 免费申请。

In [1]:
import os
from dotenv import load_dotenv
load_dotenv("../.env", override=True)
model_name = os.getenv("OPENAI_MODEL")
model_provider = os.getenv("MODEL_PROVIDER")
print(f"我们使用的模型是： {model_name}")

我们使用的模型是： qwen-plus-2025-04-28


In [2]:
from langchain.chat_models import init_chat_model
llm = init_chat_model(model_name, model_provider=model_provider, temperature=0)

## 运行模型

初始化模型后（实际上是初始化的模型供应商的API），就可以开始调用模型了。

`init_chat_model` 模块提供了标准化使用 chat models 的方法，包括：
- `invoke()`: 输入一条信息，输出一条信息；
- `stream()`: 流式输出模型产生的内容。

In [3]:
result = llm.invoke('what is agent')

In [4]:
print(type(result))
print(result.content)

<class 'langchain_core.messages.ai.AIMessage'>
The term **"agent"** can mean different things depending on the context. Here are some common definitions:

---

### 1. **General Definition**
An **agent** is a person or entity that can act on behalf of someone else (the principal), with the authority to make decisions or perform actions.

- Example: A real estate agent works on behalf of a buyer or seller to help buy or sell property.

---

### 2. **In Artificial Intelligence (AI)**
An **AI agent** is a system or program that can perceive its environment, make decisions, and take actions autonomously to achieve specific goals.

#### Types of AI Agents:
- **Simple Reflex Agents**: React based on current input.
- **Model-Based Reflex Agents**: Use internal state (memory) to make decisions.
- **Goal-Based Agents**: Make decisions to achieve certain goals.
- **Utility-Based Agents**: Choose actions that maximize utility (benefit).
- **Learning Agents**: Improve performance over time through 

In [5]:
stream_result = llm.stream('what is agent')

In [6]:
for m in llm.stream("what is agent"):
    print(m.content)


The
 term
 **
"
agent"** can
 mean different things depending
 on the context.
 Here are some common
 definitions:

---

###
 1. **
General Definition (Human
 Context)**
An
 **agent** is
 a person who acts
 on behalf of another
 person or organization,
 with the authority to
 make decisions or take
 actions.

- **
Example**: A real
 estate agent helps you
 buy or sell property
.
- **Example
**: An insurance agent
 sells insurance policies and
 represents an insurance company
.

---

### 
2. **In
 Business and Law**

An **agent**
 is someone authorized to
 act for another party
 (called the principal
) in legal or
 business matters.

-
 This is often governed
 by **agency law
**, where the agent
 has a legal duty
 to act in the
 best interest of the
 principal.

---

###
 3. **
In Artificial Intelligence (
AI)**
An
 **AI agent**
 is a software program
 or system that can
 perceive its environment,
 make decisions, and
 perform actions autonomously
 or semi-autonom
ously.

- AI
 agents 

## Tools （工具）

[Tools](https://python.langchain.com/docs/concepts/tools/) 是智能体执行任务时可以调用的工具。由LLM来决定是否使用工具，使用哪个工具，以及如何使用工具。

LLM选择好工具后，会有专门的代码来执行工具调用，并返回结果。


### 定义工具

LangChain 提供了 `@tool` 装饰器来定义工具，它会自动从函数的定义中提取函数名、描述、参数等信息。

In [7]:
from langchain.tools import tool

@tool
def write_email(to: str, subject: str, content: str) -> str:
    """撰写并发送邮件"""
    return f"邮件已发送给 {to}，主题为 {subject}，内容为 {content}"

print(type(write_email))
print(write_email.name)
print(write_email.description)
print(write_email.args)

<class 'langchain_core.tools.structured.StructuredTool'>
write_email
撰写并发送邮件
{'to': {'title': 'To', 'type': 'string'}, 'subject': {'title': 'Subject', 'type': 'string'}, 'content': {'title': 'Content', 'type': 'string'}}


### 使用工具

通过 `@tool` 装饰器定义的工具，可以被LLM使用，使用的方法是：
1. 通过 `bind_tools` 方法将工具绑定到模型上；
2. 模型会根据任务的描述，决定是否使用工具，使用哪个工具，以及具体的参数。
3. 如果使用工具，则需要在代码中执行工具函数，并获取结果（这一步与大语言模型无关）。

大模型的供应商们（如 OpenAI），还提供了一些重要的参数来控制工具的使用：
- `tool_choice`: 强制选择某个工具来执行任务，如果为 `any` 则随机选择一个工具。
- `parallel_tool_calls`: 是否使用多个工具，如果为 `true` 则使用多个工具，如果为 `False` 则最多使用一个工具。

In [8]:
model_with_tools = llm.bind_tools([write_email], tool_choice="any", parallel_tool_calls=False)

output = model_with_tools.invoke("写一封邮件给张三，主题是关于项目进展，内容是：我们正在按照计划推进项目，预计下周完成。")

In [9]:
type(output)

langchain_core.messages.ai.AIMessage

In [10]:
output

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3becfe557fa642d8ae65cf', 'function': {'arguments': '{"content": "我们正在按照计划推进项目，预计下周完成。", "subject": "项目进展", "to": "张三"}', 'name': 'write_email'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 199, 'total_tokens': 242, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen-plus-2025-04-28', 'system_fingerprint': None, 'id': 'chatcmpl-723c2fd7-2399-9fc6-8f1f-ad9043057e7f', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--ed3a4284-d3fc-44f0-a55a-fb0f64421f31-0', tool_calls=[{'name': 'write_email', 'args': {'content': '我们正在按照计划推进项目，预计下周完成。', 'subject': '项目进展', 'to': '张三'}, 'id': 'call_3becfe557fa642d8ae65cf', 'type': 'tool_call'}], usage_metadata={'input_tokens': 199, 'output_tokens': 43, 'total_tokens': 242, 'input_token_details': {}, 'output_token_details': {}})

In [11]:
# 获取工具调用的参数
args = output.tool_calls[0]['args']
args

{'content': '我们正在按照计划推进项目，预计下周完成。', 'subject': '项目进展', 'to': '张三'}

通过`invoke()`方法执行工具调用

In [12]:
result = write_email.invoke(args)
result


'邮件已发送给 张三，主题为 项目进展，内容为 我们正在按照计划推进项目，预计下周完成。'

![basic_prompt](img/tool_call.png)

通过这张图的左边部分，我们可以清晰的了解到工具调用的执行流程：
- 用户输入+Prompt+工具描述 一起输入给LLM；
- LLM决定使用哪个工具，并生成工具调用所需的参数；
- 工具调用执行，并返回结果。

通过这张图的右边部分，我们可以看出，单纯的基于Prompt的工具调用属于LLM应用的一个非常初级的形态，他的可预测性很强，但是缺少自主能力。

## Workflows (工作流)

[Workflows](https://langchain-ai.github.io/langgraph/tutorials/workflows/) 是创建LLM应用的另一种形态，它将预先定义的工作流与LLM结合起来，借助LLM的自主能力来优化工作流的执行。

![workflow_example](img/workflow_example.png)
从这张图的左边部分，可以拆解出workflow的执行流程：
- 用户输入+路由Prompt输入到第一个LLM；
- 第一个LLM判断下一步的执行步骤；
- 如果 action 是 respond，则将用户输入+Prompt+工具描述输入给第二个LLM；
- LLM决定使用哪个工具，并生成工具调用所需的参数；
- 工具调用执行，并返回结果。

从上面的流程中可以看出，通过第一个LLM的加入，增加了workflow的自主判断能力，从而使得workflow的执行更加灵活。

图片的右边部分也很形象的给出了相比于单纯的基于Prompt的工具调用，workflow有着较弱的可预测性，但是有着更强的自主能力。


## Agents (智能体)

进一步增强LLM应用的自主能力，就到了Agents。

Agents通常是通过循环调用工具来实现的，每次调用工具的结果会用来决定下一步该做什么，直到满足任务完成的条件。

![agent_example](img/agent_example.png)

相较于Workflows，Agents有着更强的自主能力，但是也更加难以预测它的执行流程。

基于这一特点，Agents更适合处理开放性的问题，没有明确的解题步骤，或者解题步骤太多，以至于很难用Workflows来实现。

Workflows 更适合那些可以提前轻松定义控制流程的场景。

![workflow_v_agent](img/workflow_v_agent.png)
